In [78]:
import numpy as np
from pyquaternion import Quaternion

def quatRotateVecDiff(q, v):
    # dq
    dq = np.hstack([np.identity(4), np.zeros([4,3])])
    # dr
    aux = np.vstack([np.zeros([1,3]), np.identity(3)])
    dr = np.hstack([np.zeros([4,4]), aux])
    # q_times_r && dq_times_r
    r_0 = np.insert(v, 0, 0)
    q_times_r = (Quaternion(q) * Quaternion(r_0)).q
    dq_times_r = quatProductDiff(q, r_0)
    # q_conj && dq_conj
    q_conj = Quaternion(q).conjugate.q
    dq_conj = quatConjugateDiff(q)
    # r_rot, dr_rot
    r_rot = (Quaternion(q_times_r) * Quaternion(q_conj)).q
    dr_rot = quatProductDiff(q_times_r, q_conj)
    # dr_rot i.e., res
    aux = np.vstack([dq, dr])
    aux = np.dot(dq_times_r, aux)
    aux = np.vstack([aux, np.dot(dq_conj, dq)])
    dr_rot = np.dot(dr_rot, aux)
    res = dr_rot[1:]
    return res

def quatProductDiff(lhs, rhs):
    # w1
    w1 = lhs[0]
    # w2
    w2 = rhs[0]
    # v1
    v1 = lhs[1:].reshape(-1, 1)
    # v2
    v2 = rhs[1:].reshape(-1, 1)
    
    # dq1
    dq1 = np.hstack([np.identity(4), np.zeros([4,4])])
    # dq2
    dq2 = np.hstack([np.zeros([4,4]), np.identity(4)])
    # dw1
    dw1 = dq1[0].reshape(1, -1)
    # dw2
    dw2 = dq2[0].reshape(1, -1)
    # dv1
    dv1 = dq1[1:]
    # dv2
    dv2 = dq2[1:]
    
    # dw3
    dw3 = w2*dw1 + w1*dw2 - np.dot(v2.T, dv1) - np.dot(v1.T, dv2)
    # dv3
    dv3 = dcross(v1, v2, dv1, dv2) + np.dot(v2, dw1) + w1*dv2 + np.dot(v1, dw2) + w2*dv1
    
    # dq3
    dq3 = np.vstack([dw3, dv3])
    
    return dq3

def quatConjugateDiff(q):
    res = -np.identity(4)
    res[0][0] = 1
    return res

def dcross(r1, r2, dr1, dr2):
    r1 = r1.flatten()
    r2 = r2.flatten()
    lhs = np.zeros([3,8])
    rhs = np.zeros([3,8])
    for i in xrange(8):
        lhs[:,i] = np.cross(r1, dr2[:,i])
        rhs[:,i] = np.cross(r2, dr1[:,i])
    return lhs-rhs
    
    

q = np.array([1,2,3,4])
q = q/np.linalg.norm(q)
v = np.array([4,5,6])
print q
print quatRotateVecDiff(q,v)

[ 0.18257419  0.36514837  0.54772256  0.73029674]
[[  0.73029674  17.16197347   1.46059349  -3.28633535  -0.66666667
    0.13333333   0.73333333]
 [  3.28633535  -1.46059349  17.16197347   0.73029674   0.66666667
   -0.33333333   0.66666667]
 [  1.46059349   3.28633535  -0.73029674  17.16197347   0.33333333
    0.93333333   0.13333333]]
